In [ ]:
''' Function to find the degree of each vertex and return a sorted dictionary'''
def vertex_degrees(graph):
    degree_dict = {}  # dictionary to store degrees of each node

    # count the number of connections for each node
    for node in graph:
        count = 0
        for neighbour in graph[node]:
            count += 1
        degree_dict[node] = count

    # now we will sort the nodes based on their degree (smallest to largest)
    keys = []  # list to store all nodes
    for node in degree_dict:
        keys.append(node)

    # bubble sort based on degree values
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            if degree_dict[keys[i]] > degree_dict[keys[j]]:
                # swap nodes
                temp = keys[i]
                keys[i] = keys[j]
                keys[j] = temp
    # now build a new dictionary with sorted nodes
    sorted_degree_dict = {}
    for node in keys:
        sorted_degree_dict[node] = degree_dict[node]
    return sorted_degree_dict


'''Convert adjacency list to adjacency matrix'''
def adj_list_to_matrix(graph):
    nodes = list(graph.keys())  # List of all nodes
    index = {node: i for i, node in enumerate(nodes)}  # Mapping node → index for matrix position
    size = len(nodes)
    matrix = [[0]*size for _ in range(size)]  # Create a square matrix filled with 0s
    for node in graph:
        for neighbor in graph[node]:
            matrix[index[node]][index[neighbor]] = 1  # Mark 1 where an edge exists
    return matrix, nodes

''' Convert adjacency matrix to adjacency list'''
def matrix_to_adj_list(matrix, nodes):
    adj_list = {node: [] for node in nodes}  # Create empty list for each node
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 1:
                adj_list[nodes[i]].append(nodes[j])  # Add neighbor if there's an edge
    return adj_list

'''Convert edge list to adjacency list'''
def edge_list_to_adj_list(edges):
    adj_list = {}  # this will store the final adjacency list

    for edge in edges:
        u = edge[0]
        v = edge[1]

        # if u is not in the adj_list, create an empty list for it
        if u not in adj_list:
            adj_list[u] = []
        # if v is not in the adj_list, create an empty list for it
        if v not in adj_list:
            adj_list[v] = []

        # add v to u's list
        adj_list[u].append(v)
        # since the graph is undirected, also add u to v's list
        adj_list[v].append(u)

    return adj_list


'''Check if two vertices are adjacent '''
def are_adjacent(graph, u, v):
    # First we will check if 'u' is a key in the graph
    if u in graph:
        # Then we check if 'v' is present in u's list of neighbors
        for neighbor in graph[u]:
            if neighbor == v:
                return True  # they are connected
    return False  # either u is not in graph or v is not in u's list


'''Check if graph is complete (every node connected to every other node)'''
def is_complete(graph):
    n = len(graph)
    for node in graph:
        if len(graph[node]) != n - 1:  # In a complete graph, degree = total_nodes - 1
            return False
    return True

'''check if graph is connected (all nodes reachable from any node)'''
def is_connected(graph):
    visited = []  # list to keep track of visited nodes
    queue = []    # list to use as queue 

    # pick the first node from the graph (arbitrary start)
    for node in graph:
        start = node
        break

    queue.append(start)

    # loop to go through all connected nodes
    while len(queue) > 0:
        current_node = queue[0]     # get the first element
        queue = queue[1:]           # remove it from the queue
        if current_node not in visited:
            visited.append(current_node)
            # go through all neighbours of the current node
            for neighbour in graph[current_node]:
                if neighbour not in visited and neighbour not in queue:
                    queue.append(neighbour)
    # if number of visited nodes is equal to total nodes, the graph is connected
    if len(visited) == len(graph):
        return True
    else:
        return False    # Graph is connected if all nodes were visited

''' Classify a list of vertices as walk, trail, path or none'''
def classify_walk(graph, vertices):
    edges_used = set()  # Track edges used
    seen_nodes = set()  # Track visited nodes
    is_trail = True     # No edge repetition
    is_path = True      # No vertex repetition

    for i in range(len(vertices) - 1):
        u, v = vertices[i], vertices[i+1]
        if v not in graph.get(u, []):  # Not a valid walk if edge doesn't exist
            return "None"
        if (u, v) in edges_used or (v, u) in edges_used:
            is_trail = False  # Edge repeated
        else:
            edges_used.add((u, v))
        if v in seen_nodes:
            is_path = False  # Vertex repeated
        seen_nodes.add(u)
    seen_nodes.add(vertices[-1])  # Add last node to set

    if is_path:
        return "Path"
    elif is_trail:
        return "Trail"
    else:
        return "Walk"

''' Check if a graph is a tree '''
def is_tree(graph):
    visited = []  # This will keep track of visited nodes
    # A simple DFS function to go through the graph
    def dfs(node, parent):
        visited.append(node)  # Mark this node as visited
        # Go through all connected nodes (neighbors)
        for neighbor in graph[node]:
            if neighbor not in visited:
                # If not visited, keep going deeper
                if dfs(neighbor, node) == False:
                    return False
            elif neighbor != parent:
                # If we visit a node that was already visited and it's not the parent, cycle found
                return False
        return True  # No problem, keep going
    # Get the first node to start DFS (we assume graph has at least one node)
    for key in graph:
        start = key
        break
    if dfs(start, None) == False:
        return False  # If we find a cycle
    # After DFS, if not all nodes are visited, then graph is not connected
    if len(visited) != len(graph):
        return False
    return True  # No cycles and all nodes connected = it's a tree


''' Function to get the spanning tree of a graph'''
def get_spanning_tree(graph):
    tree = {}         # This will store the spanning tree
    visited = []      # A list to keep track of visited nodes
    queue = []        # A list used as a queue

    # Start from the first node in the graph
    for key in graph:
        start = key  # Set the start node to the first key in the graph
        break

    # Add the start node to the queue and mark it as visited
    queue.append(start)
    visited.append(start)

    # While there are nodes in the queue to process
    while len(queue) > 0:
        node = queue[0]  # Get the first node from the queue 
        queue = queue[1:]  # Remove the first node from the queue (pop from front)

        # Look at each neighbor of the current node
        for neighbor in graph[node]:
            if neighbor not in visited:  # If this neighbor hasn't been visited yet
                visited.append(neighbor)  # Mark this neighbor as visited

                # If the node is not in the tree yet, add it with an empty list
                if node not in tree:
                    tree[node] = []
                if neighbor not in tree:
                    tree[neighbor] = []

                # Add the edge to the tree (undirected graph, so add both directions)
                tree[node].append(neighbor)
                tree[neighbor].append(node)

                # Add the neighbor to the queue for further processing
                queue.append(neighbor)

    return tree  # Return the spanning tree



'''Count how many leaf nodes in a tree '''
def count_leaf_nodes(tree):
    # Start a counter for leaf nodes
    count = 0
    # Go through each node in the tree
    for node in tree:
        # If the node has only one connection (child), it is a leaf node
        if len(tree[node]) == 1:
            count += 1  # Increase the leaf node count by 1
    # Return the total number of leaf nodes
    return count


'''Check if a tree is binary '''
def is_binary_tree(tree):
    for node in tree:
        if len(tree[node]) > 2:
            return False
    return True

'''Find the height of a node (longest path from node to any leaf)'''
def find_height(tree, node):
    # If the node has no children, it's a leaf, and its height is 0
    if len(tree[node]) == 0:
        return 0
    # Initialize a variable to keep track of the maximum height found among children
    max_height = -1
    # Loop through all children and calculate their heights
    for child in tree[node]:
        child_height = find_height(tree, child)  # Recursively find height of each child
        max_height = max(max_height, child_height)  # Keep the largest height
    # The height of the current node is 1 + the maximum height of its children
    return max_height + 1

''' Find the depth of a node from the root (distance from root to target node)'''
def find_depth(tree, target, current_depth=0):
    # Start the search from the root node (assumed to be the first key in the tree)
    root = list(tree.keys())[0]  # We assume the first node is the root
    
    # If the target is found at the current root node, return the current depth
    if root == target:
        return current_depth
    
    # Recursively search each child of the root node
    for child in tree[root]:
        result = find_depth(tree, target, current_depth + 1)
        if result != -1:  # If the target node is found in one of the child branches, return the depth
            return result
    # If the target is not found, return -1
    return -1


